#LIBRARY IMPORT

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

#필요한 함수정의

In [2]:
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

#데이터 불러오기

In [3]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/LG AI대회/'

Mounted at /content/drive


In [4]:
#데이터 모두 불러옴

train_quality  = pd.read_csv(PATH+'train_quality_data.csv')

test_quality  = pd.read_csv(PATH+'test_quality_data.csv')

#TRAIN 변수 만들기(DataFrame)

In [5]:
quality = train_quality
quality.info()
quality.fillna(0) #quality_5변수에 NA값이 있음

#quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
        'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
quality[cols] = quality[cols].replace({',':''}, regex= True) #2,345이런식으로 표현된(문자열) 원소들이 있음
quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [8]:
#quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

#minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
#현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
minusQ = []
for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
  if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
  #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
    minusQ.append(-1)
  else:
    minusQ.append(0)

quality['minusQ'] = minusQ #minusQ 열 추가해주기

In [9]:
#규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
#minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

def func(x):
    d = {}
    d['quality_0'] = x['quality_0'].max()
    d['quality_1'] = x['quality_1'].max()
    d['quality_2'] = x['quality_2'].max()
    d['quality_3'] = x['quality_3'].max()
    d['quality_4'] = x['quality_4'].max()
    d['quality_5'] = x['quality_5'].max()
    d['quality_6'] = x['quality_6'].max()
    d['quality_7'] = x['quality_7'].max()
    d['quality_8'] = x['quality_8'].max()
    d['quality_9'] = x['quality_9'].max()
    d['quality_10'] = x['quality_10'].max()
    d['quality_11'] = x['quality_11'].max()
    d['quality_12'] = x['quality_12'].max()
    d['minusQ'] = x['minusQ'].sum()
    return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

quality5 = quality.groupby(['user_id','time']).apply(func)
quality5

quality_0  quality_1  ...  quality_12  minusQ
user_id time                                  ...                    
10000   20201129090000        0.0        0.0  ...         0.0     0.0
        20201130210000        0.0        0.0  ...         0.0     0.0
10002   20201104110000        2.0        0.0  ...         0.0     0.0
        20201106010000        0.0        0.0  ...         0.0     0.0
        20201111010000        0.0        0.0  ...         0.0     0.0
...                           ...        ...  ...         ...     ...
24993   20201130203000        0.0        0.0  ...         0.0    -3.0
24995   20201128202000        0.0        0.0  ...         0.0    -8.0
        20201129002000        0.0        0.0  ...         0.0    -2.0
24997   20201108230000        0.0        0.0  ...         0.0    -4.0
        20201124033000        0.0        0.0  ...         0.0     0.0

[68952 rows x 14 columns]

In [10]:
quality5 = quality5.groupby(['user_id']).sum()
quality5 = quality5.drop(columns = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 'quality_6']) 
#아래의 특징에 근거하여 일부 quality변수제거(이 외에도 다양한 경우로 조합이 가능할듯)
#quality 0,1,2,5,6,11 은 함께 -1을 가지며 발현되는 경우가 매우 많음
#quality 0와 2이 매우 비슷함
#quality 1과 11이 매우 비슷함(0과 2만큼은 아님)
#quality 3과 4는 모두 0
#quality 6의 누적합은 7
#quality 5의 누적합은 10
#quality 12는 대체로 0이며 큰 숫자가 없음
#quality 8은 아예 모르겠음

quality5

,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ
user_id,,,,,,,
10000,0.0,0.0,0.0,12.0,0.0,0.0,0.0
10002,46.0,0.0,1.0,31.0,0.0,0.0,-2.0
10004,87.0,0.0,0.0,4.0,0.0,0.0,-2.0
10005,36.0,0.0,0.0,10.0,0.0,0.0,-10.0
10006,4.0,0.0,0.0,12.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
24990,0.0,0.0,0.0,2.0,0.0,0.0,-8.0
24992,0.0,0.0,0.0,19.0,0.0,0.0,-12.0
24993,0.0,0.0,0.0,8.0,0.0,0.0,-7.0


In [11]:
print(quality5.shape)

#추후에left조인(by: user_id)을 이용하고 fillna(0)을 이용

(8281, 7)


#TEST변수 만들기(DataFrame)

In [12]:
test_quality  = pd.read_csv(PATH+'test_quality_data.csv')


quality = test_quality
quality.info()
quality.fillna(0) #quality_5변수에 NA값이 있음

#quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
        'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
quality[cols] = quality[cols].replace({',':''}, regex= True) #2,345이런식으로 표현된(문자열) 원소들이 있음
quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기


#quality6만들기(quality의 일부 + -1의 개수 뽑아냄)

minusQ = []
for i in range(0,len(quality)):
  if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
  #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
    minusQ.append(-1)
  else:
    minusQ.append(0)
print(pd.Series(minusQ).value_counts())

quality['minusQ'] = minusQ
#print(quality)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        747972 non-null  int64  
 1   user_id     747972 non-null  int64  
 2   fwver       725208 non-null  object 
 3   quality_0   641388 non-null  float64
 4   quality_1   747961 non-null  object 
 5   quality_2   726857 non-null  float64
 6   quality_3   747972 non-null  int64  
 7   quality_4   747972 non-null  int64  
 8   quality_5   747928 non-null  object 
 9   quality_6   747972 non-null  int64  
 10  quality_7   747972 non-null  object 
 11  quality_8   747972 non-null  object 
 12  quality_9   747972 non-null  object 
 13  quality_10  747972 non-null  object 
 14  quality_11  747972 non-null  int64  
 15  quality_12  747972 non-null  int64  
dtypes: float64(2), int64(7), object(7)
memory usage: 91.3+ MB
 0    606018
-1    141954
dtype: int64


In [13]:
def func(x):
    d = {}
    d['quality_0'] = x['quality_0'].max()
    d['quality_1'] = x['quality_1'].max()
    d['quality_2'] = x['quality_2'].max()
    d['quality_3'] = x['quality_3'].max()
    d['quality_4'] = x['quality_4'].max()
    d['quality_5'] = x['quality_5'].max()
    d['quality_6'] = x['quality_6'].max()
    d['quality_7'] = x['quality_7'].max()
    d['quality_8'] = x['quality_8'].max()
    d['quality_9'] = x['quality_9'].max()
    d['quality_10'] = x['quality_10'].max()
    d['quality_11'] = x['quality_11'].max()
    d['quality_12'] = x['quality_12'].max()
    d['minusQ'] = x['minusQ'].sum()
    return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

 
quality6 = quality.groupby(['user_id','time']).apply(func)
quality6


quality6 = quality6.groupby(['user_id']).sum()
quality6 = quality6.drop(columns = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 'quality_6'])
quality6

,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ
user_id,,,,,,,
30000,5.0,0.0,0.0,2.0,0.0,0.0,0.0
30001,0.0,0.0,0.0,2563.0,0.0,0.0,-9.0
30002,0.0,0.0,0.0,133.0,0.0,0.0,-15.0
30003,220.0,0.0,0.0,12.0,0.0,0.0,0.0
30004,33.0,0.0,0.0,2.0,0.0,0.0,-6.0
...,...,...,...,...,...,...,...
44990,0.0,0.0,0.0,50.0,0.0,0.0,-4.0
44993,109.0,0.0,9.0,14.0,0.0,0.0,-2.0
44994,0.0,0.0,0.0,2.0,0.0,0.0,-11.0


In [15]:
print(quality6.shape)

#추후에left조인(by: user_id)을 이용하고 fillna(0)을 이용

(8268, 7)
